# 阶跃星辰(StepFun)视频理解最佳实践
## 概述
本 notebook 基于 [阶跃星辰视频理解开发指南](https://platform.stepfun.com/docs/guide/video_chat) 编写，涵盖 **基础视频理解调用、Files API 加速、ffmpeg 处理大文件/格式转换** 等核心最佳实践，帮助开发者高效实现视频内容分析与生成任务。

### 核心目标
- 掌握 step-1o-turbo-vision 模型的视频理解调用流程
- 利用 Files API 优化视频加载速度，降低重复流量消耗
- 通过 ffmpeg 处理超大/非 MP4 格式视频，适配模型要求
- 理解价格影响因素与性能优化技巧

## 1. 环境准备
### 1.1 安装依赖库
需安装 `requests` 用于 API 调用，`python-dotenv` 管理环境变量。

In [3]:

# 导入必要库
import requests
import os
import subprocess

### 1.2 配置环境变量
将阶跃星辰 API Key 存储在 `.env` 文件中（格式：`STEP_API_KEY=你的密钥`），避免硬编码密钥。
> 提示：API Key 需从阶跃星辰开发者平台获取。

In [4]:

# 获取 API Key
API_KEY = os.getenv("STEPFUN_API_KEY")
if not API_KEY:
    raise ValueError("请在环境变量中设置 STEPFUN_API_KEY")

# 基础配置
BASE_URL = "https://api.stepfun.com/v1"
RECOMMENDED_MODEL = "step-1o-turbo-vision"  # 推荐模型

### 1.3 安装 ffmpeg（可选，处理视频时需）
- Windows：从 [FFmpeg 官网](https://ffmpeg.org/download.html) 下载，添加至系统环境变量
- macOS：`brew install ffmpeg`
- Linux：`sudo apt update && sudo apt install ffmpeg`

验证安装是否成功：

In [ ]:
# 验证 ffmpeg 安装
try:
    subprocess.run(["ffmpeg", "-version"], capture_output=True, check=True)
    print("✅ ffmpeg 安装成功")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("❌ ffmpeg 未安装，请参考上文安装步骤")

✅ ffmpeg 安装成功


## 2. 基础视频理解调用（入门实践）
### 核心要求
- 视频格式：仅支持 **MP4**
- 视频来源：可直接访问的网络 URL（暂不支持本地文件上传）
- 模型选择：优先使用 `step-1o-turbo-vision`（更强性能、更低费用）

In [ ]:
def basic_video_understanding(video_url: str, prompt: str) -> str:
    # 构造请求体
    payload = {
        "model": RECOMMENDED_MODEL,
        "messages": [
            {
                "role": "user",
                "content": [
                    # 最佳实践：视频信息放在指令前，提升模型效果
                    {
                        "type": "video_url",
                        "video_url": {"url": video_url}
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": 1024  # 控制生成文本长度
    }

    # 发送请求
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    try:
        print(f"🔄 正在调用 {RECOMMENDED_MODEL} 模型处理视频...")
        response = requests.post(
            f"{BASE_URL}/chat/completions",
            json=payload,
            headers=headers,
            timeout=60  # 视频处理耗时较长，设置合理超时
        )
        response.raise_for_status()  # 抛出 HTTP 错误
        result = response.json()
        print("traceid:",response.id.split('.', 1)[0])
        return result["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        return f"❌ 调用失败：{str(e)}"


# 示例：基于视频生成诗意文案（使用官方示例视频）
SAMPLE_VIDEO_URL = "https://static.stepfun.com/static/platform-web/vipcase/case1.mp4"
PROMPT = (
    "你是文案专家，通过分析视频中的风景和拍摄风格，创作极简且充满诗意的文案。"
)

# 执行调用
output = basic_video_understanding(SAMPLE_VIDEO_URL, PROMPT)
print("\n🎯 模型生成结果：")
print(output)

🔄 正在调用 step-1o-turbo-vision 模型处理视频...

🎯 模型生成结果：
在落日余晖中，  
她将头轻轻靠在他的肩，  
两杯酒，映着天边的云，  
静默，便是最好的陪伴。


## 3. 最佳实践一：使用 Files API 加速视频理解
### 适用场景
当视频需要 **重复使用**（如 Few-shot 示例）时，将视频上传至阶跃星辰文件存储，避免重复下载，提升速度并减少流量消耗。

### 实现步骤
1. 调用 Files API 上传视频（purpose=storage）
2. 获取 File ID，拼接为 `stepfile://{file_id}` 格式
3. 使用拼接后的地址进行视频理解调用

In [7]:
def upload_video_to_stepfun(file_path: str) -> str:
    # 验证文件格式
    if not file_path.endswith(".mp4"):
        return "❌ 仅支持 MP4 格式视频上传"

    # 验证文件存在
    if not os.path.exists(file_path):
        return f"❌ 文件不存在：{file_path}"

    # 构造上传请求
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    with open(file_path, "rb") as f:
        files = {
            "file": (os.path.basename(file_path), f),
            "purpose": (None, "storage")  # 必须设置为 storage
        }

        try:
            print(f"📤 正在上传视频：{os.path.basename(file_path)}")
            response = requests.post(
                f"{BASE_URL}/files",
                headers=headers,
                files=files,
                timeout=120
            )
            response.raise_for_status()
            file_id = response.json()["id"]
            stepfile_url = f"stepfile://{file_id}"
            print(f"✅ 上传成功，stepfile 地址：{stepfile_url}")
            return stepfile_url
        except requests.exceptions.RequestException as e:
            return f"❌ 上传失败：{str(e)}"


# 示例：上传本地视频并使用其进行理解
LOCAL_VIDEO_PATH = "./media/04_autumn.mp4"  # 替换为你的本地 MP4 路径

# 1. 上传视频获取 stepfile 地址
stepfile_url = upload_video_to_stepfun(LOCAL_VIDEO_PATH)

# 2. 使用 stepfile 地址进行视频理解（复用基础调用函数）
if stepfile_url.startswith("stepfile://"):
    output = basic_video_understanding(stepfile_url, PROMPT)
    print("\n🎯 基于 stepfile 的生成结果：")
    print(output)

📤 正在上传视频：04_autumn.mp4
✅ 上传成功，stepfile 地址：stepfile://file-M7enJ4HVzc
🔄 正在调用 step-1o-turbo-vision 模型处理视频...

🎯 基于 stepfile 的生成结果：
1. **音乐的秋天**  
绿企鹅轻声哼唱，猫猫悄悄和鸣，旋律在角落流淌，秋天的乐章悄然响起。

2. **食欲的秋天**  
美食当前，汗珠微落，满足的秋日，藏在心底的温暖。

3. **运动的秋天**  
阳光下，身影跃动，汗水与笑容，是秋天的活力诗篇。

4. **艺术的秋天**  
画笔轻舞，色彩斑斓，伙伴的笑颜，在画布上绽放。  
金奖的荣耀，是秋天最美的礼物。


## 4. 最佳实践二：用 ffmpeg 处理超大/非 MP4 视频
### 适用场景
- 视频超过 128MB（step-1.5v-mini 限制，step-1o-turbo-vision 建议拆分提升效率）
- 视频格式非 MP4（如 MKV、AVI 等）

In [14]:

#将非 MP4 视频转换为 MP4 格式（无损拷贝编码）
def convert_video_to_mp4(input_path: str, output_path: str = None) -> str:
    if not output_path:
        output_path = os.path.splitext(input_path)[0] + ".mp4"

    cmd = [
        "ffmpeg",
        "-y",             # 自动覆盖已存在的文件
        "-i", input_path,
        "-codec", "copy",  # 无损拷贝，速度快
        output_path
    ]

    try:
        print(f"🔄 正在转换 {input_path} 为 MP4...")
        subprocess.run(cmd, capture_output=True, check=True, text=True)
        return f"✅ 转换成功：{output_path}"
    except subprocess.CalledProcessError as e:
        return f"❌ 转换失败：{e.stderr}"


# 将MP4大视频切割为多个指定时长的小视频（无损切割）
def split_large_video(input_path: str, segment_seconds: int = 120) -> str:
    if not input_path.endswith(".mp4"):
        return "❌ 仅支持 MP4 格式视频切割，请先转换"

    output_pattern = "./media/04_cut_%d.mp4"    # 输出文件名格式
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-acodec", "copy",  # 音频流无损拷贝
        "-f", "segment",
        "-segment_time", str(segment_seconds),  # 片段时长
        "-vcodec", "copy",  # 视频流无损拷贝
        "-reset_timestamps", "1",  # 重置每个片段的时间戳
        "-map", "0",  # 映射所有流
        output_pattern
    ]

    try:
        print(f"🔄 正在切割视频，每段 {segment_seconds} 秒...")
        subprocess.run(cmd, capture_output=True, check=True, text=True)
        return f"✅ 切割成功，输出文件：{output_pattern}"
    except subprocess.CalledProcessError as e:
        return f"❌ 切割失败：{e.stderr}"


# 示例 1：转换其他格式视频为 MP4
mkv_video_path = "./media/04_sample.mov"  
if os.path.exists(mkv_video_path):
    print(convert_video_to_mp4(mkv_video_path))

# 示例 2：切割大视频为 120 秒片段 (这里示例切为20s)
large_video_path = "./media/04_autumn.mp4"  
if os.path.exists(large_video_path):
    print(split_large_video(large_video_path, segment_seconds=40))

🔄 正在转换 ./media/04_sample.mov 为 MP4...
✅ 转换成功：./media/04_sample.mp4
🔄 正在切割视频，每段 40 秒...
✅ 切割成功，输出文件：./media/04_cut_%d.mp4


## 5. 切割后视频的整合处理
对于切割后的多个视频片段，可逐段调用 API 生成结果，再通过模型整合为完整结论。

In [15]:
#    处理切割后的视频片段，整合生成结果
def process_split_videos(segment_pattern: str, prompt: str) -> str:
    # 收集所有片段文件
    segments = []
    i = 0
    while True:
        segment_path = segment_pattern % i
        if os.path.exists(segment_path):
            segments.append(segment_path)
            i += 1
        else:
            break

    if not segments:
        return "未找到视频片段"
    
    print(f"找到 {len(segments)} 个视频片段，开始逐段处理...")
    segment_results = []

    # 逐段上传并处理
    for idx, seg_path in enumerate(segments):
        print(f"\n--- 处理片段 {idx+1}/{len(segments)}: {os.path.basename(seg_path)} ---")
        seg_stepfile = upload_video_to_stepfun(seg_path)
        if seg_stepfile.startswith("stepfile://"):
            res = basic_video_understanding(seg_stepfile, prompt)
            print("处理结果:",res)
            segment_results.append(f"片段 {idx+1}：{res}")

    # 整合结果
    integrate_prompt = (
        f"以下是视频各片段的分析结果，请将其整合为一段连贯、完整的内容，忽略片段标记：\n"
        + "\n".join(segment_results)
    )

    # 调用模型整合
    integrate_result = basic_video_understanding(
        video_url=seg_stepfile,  # 用任意片段的 stepfile 占位（仅需文本整合）
        prompt=integrate_prompt
    )

    return f"整合后结果：\n{integrate_result}"


# 示例：处理切割后的视频片段
SEGMENT_PATTERN = "./media/04_cut_%d.mp4"  # 与切割时的输出pattern一致
if os.path.exists(SEGMENT_PATTERN % 0):
    final_result = process_split_videos(SEGMENT_PATTERN, PROMPT)
    print(final_result)

找到 2 个视频片段，开始逐段处理...

--- 处理片段 1/2: 04_cut_0.mp4 ---
📤 正在上传视频：04_cut_0.mp4
✅ 上传成功，stepfile 地址：stepfile://file-M7gqbSPDxg
🔄 正在调用 step-1o-turbo-vision 模型处理视频...
处理结果: 在角落的生活中，  
音乐轻扬，秋日呢喃。  
贪恋美食，汗滴轻落，  
梦里运动，笑看慵懒。

--- 处理片段 2/2: 04_cut_1.mp4 ---
📤 正在上传视频：04_cut_1.mp4
✅ 上传成功，stepfile 地址：stepfile://file-M7grxiPxLc
🔄 正在调用 step-1o-turbo-vision 模型处理视频...
处理结果: 在秋日的温柔里，  
画笔轻舞，心随色彩游。  
伙伴们的笑，  
在画布上，开出金奖的花。  
角落的生物，  
梦在电影院延续。
🔄 正在调用 step-1o-turbo-vision 模型处理视频...
整合后结果：
在角落的生活中，音乐轻扬，秋日呢喃。贪恋美食，汗滴轻落，梦里运动，笑看慵懒。在秋日的温柔里，画笔轻舞，心随色彩游。伙伴们的笑，在画布上，开出金奖的花。角落的生物，梦在电影院延续。

视频中，五只可爱的角落生物围坐在桌子旁画画，白熊戴着贝雷帽，披着红色斗篷，询问大家是否画完了。绿色的企鹅画了一根青椒，咖啡色的团子画了一个酱油瓶，黄色的猫咪画了鱼和章鱼小丸子，蓝色的恐龙画了两条鱼和一个星星，绿色的叶子状生物画了一幅杂乱的画，上面画满了各种伙伴的简笔画。白熊给叶子状生物颁发了金奖，所有角色都为它鼓掌庆祝。最后，画面切换到电影院的宣传，表示电影也在热映中。


## 6. 价格预估与性能优化总结
### 6.1 价格影响因素
视频理解费用由 **Prompt 长度** 和 **视频长度** 决定，视频越长、分辨率越高，输入 Token 越多，费用越高。参考示例：

| 视频分辨率 | 视频长度 | 输入 Token | 预估输入价格 |
|------------|----------|------------|--------------|
| 3840x2160  | 00:14    | 5238       | 0.013095 元  |
| 4096x2160  | 01:02    | 24064      | 0.06016 元   |

### 6.2 核心优化技巧
1. **模型选择**：强制使用 `step-1o-turbo-vision`，避免使用即将停用的 `step-1.5v-mini`
2. **视频预处理**：
   - 非 MP4 格式用 ffmpeg 转换为 MP4
   - 大视频切割为 120 秒内片段，提升处理速度
3. **存储优化**：重复使用的视频通过 Files API 上传至阶跃星辰存储，减少下载耗时与流量
4. **交互设计**：视频处理耗时较长，需在产品中添加等待提示（如加载动画），降低用户焦虑
5. **视频托管**：将视频放在 CDN 或高带宽对象存储，提升下载速度

## 7. 关键注意事项
- **视频位置**：务必将 `video_url` 放在 `text` 指令前，模型效果更优
- **模型限制**：`step-1.5v-mini` 不支持 tool_call，且仅支持 ≤128MB 的 MP4
- **超时设置**：视频下载+处理耗时较长，API 调用超时建议设置为 60-120 秒
- **错误处理**：需捕获 HTTP 错误（如 401 密钥无效、404 视频无法访问）

## 8. 总结
本 notebook 覆盖了阶跃星辰视频理解的全流程最佳实践，包括：
1. 基础 API 调用入门
2. Files API 加速技巧
3. ffmpeg 视频预处理方案
4. 切割视频的整合处理

开发者可根据实际场景（如视频大小、复用频率）选择对应的优化方案，以实现高效、低成本的视频理解任务。